In [68]:
import pandas as pd
import geopandas as gpd
from census import Census
from us import states

c = Census("771ac44f4c55670ec890af7951073be949e5130c")

https://pygis.io/docs/d_access_census.html

https://api.census.gov/data/2020/acs/acs5/variables.html

https://api.census.gov/data/2020/acs/acs5/subject/variables.html

https://unicede.air-worldwide.com/unicede/unicede_washington_fips.html

In [69]:
COUNTIES = ("001", "003", "019", "023", "043", "051", "063", "065", "075")


In [70]:
acs5_data = c.acs5.state_county_tract(
    fields=(
        "NAME",
        "B28003_002E",
        "B28011_002E",
        "B01003_001E",
        "B08101_049E",
        "B28002_004E",
        "B01002_001E",
        
    ),
    state_fips=states.WA.fips,
    county_fips="*",
    tract="*",
    year=2020,
)


In [71]:
acs5s_data = c.acs5st.state_county_tract(
    fields=(
        "NAME",
        "S1901_C01_012E",
    ),
    state_fips=states.WA.fips,
    county_fips="*",
    tract="*",
    year=2020,
)


In [72]:
variable_data = acs5_data + acs5s_data


In [73]:
acs5_data = pd.DataFrame(acs5_data)
acs5s_data = pd.DataFrame(acs5s_data)


In [74]:
data = pd.merge(acs5_data, acs5s_data)

data["GEOID"] = data.state + data.county + data.tract

data = data[
    [
        "NAME",
        "GEOID",
        "tract",
        "B28003_002E",
        "B28011_002E",
        "B08101_049E",
        "B28002_004E",
        "S1901_C01_012E",
        "B01002_001E",
        "state",
        "county",
        "B01003_001E",
    ]
]

data = data[data.county.isin(COUNTIES)].drop(columns=["state", "county", "tract"])

data.head()


,NAME,GEOID,B28003_002E,B28011_002E,B08101_049E,B28002_004E,S1901_C01_012E,B01002_001E,B01003_001E
0,"Census Tract 9601, Asotin County, Washington",53003960100,1664.0,1569.0,145.0,1561.0,73942.0,50.1,4320.0
1,"Census Tract 9602, Asotin County, Washington",53003960200,1872.0,1780.0,421.0,1753.0,72988.0,53.9,4755.0
2,"Census Tract 9603, Asotin County, Washington",53003960300,1279.0,1156.0,6.0,1156.0,38077.0,34.5,3633.0
3,"Census Tract 9604, Asotin County, Washington",53003960400,941.0,805.0,29.0,732.0,38777.0,38.9,2517.0
4,"Census Tract 9605, Asotin County, Washington",53003960500,1345.0,1198.0,26.0,1192.0,42639.0,36.8,3517.0


In [75]:
data.columns.tolist()


['NAME',
 'GEOID',
 'B28003_002E',
 'B28011_002E',
 'B08101_049E',
 'B28002_004E',
 'S1901_C01_012E',
 'B01002_001E',
 'B01003_001E']

In [76]:
data = data.rename(
    columns={
        "B28003_002E": "has_computer",
        "B28011_002E": "with_internet",
        "B01003_001E": "total_population",
        "B08101_049E": "work_from_home",
        "S1901_C01_012E": "median_income",
        "B28002_004E": "broadband",
        "B01002_001E": "median_age",
    }
)

In [77]:
data.isna().any()

NAME                False
GEOID               False
has_computer        False
with_internet       False
work_from_home      False
broadband           False
median_income       False
median_age          False
total_population    False
dtype: bool

In [78]:
data.head()

,NAME,GEOID,has_computer,with_internet,work_from_home,broadband,median_income,median_age,total_population
0,"Census Tract 9601, Asotin County, Washington",53003960100,1664.0,1569.0,145.0,1561.0,73942.0,50.1,4320.0
1,"Census Tract 9602, Asotin County, Washington",53003960200,1872.0,1780.0,421.0,1753.0,72988.0,53.9,4755.0
2,"Census Tract 9603, Asotin County, Washington",53003960300,1279.0,1156.0,6.0,1156.0,38077.0,34.5,3633.0
3,"Census Tract 9604, Asotin County, Washington",53003960400,941.0,805.0,29.0,732.0,38777.0,38.9,2517.0
4,"Census Tract 9605, Asotin County, Washington",53003960500,1345.0,1198.0,26.0,1192.0,42639.0,36.8,3517.0


In [82]:
data[["tract", "county", "state"]] = data.NAME.str.split(",", expand=True)

In [84]:
data.drop(columns=["NAME"], inplace=True)

In [85]:
data.head()

,GEOID,has_computer,with_internet,work_from_home,broadband,median_income,median_age,total_population,tract,county,state
0,53003960100,1664.0,1569.0,145.0,1561.0,73942.0,50.1,4320.0,Census Tract 9601,Asotin County,Washington
1,53003960200,1872.0,1780.0,421.0,1753.0,72988.0,53.9,4755.0,Census Tract 9602,Asotin County,Washington
2,53003960300,1279.0,1156.0,6.0,1156.0,38077.0,34.5,3633.0,Census Tract 9603,Asotin County,Washington
3,53003960400,941.0,805.0,29.0,732.0,38777.0,38.9,2517.0,Census Tract 9604,Asotin County,Washington
4,53003960500,1345.0,1198.0,26.0,1192.0,42639.0,36.8,3517.0,Census Tract 9605,Asotin County,Washington


In [94]:
tracts = data.tract.str.split(" ", expand=True).rename(columns={2: "tract"}).tract

In [97]:
data = pd.concat([tracts, data], axis=1).drop(columns=["state"])

In [98]:
data.to_csv("../app/data/census_data.csv", index=False)